In [ ]:
!pip install -q py_vncorenlp

import os
import py_vncorenlp
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import re

In [ ]:
# Ensure the directory exists before downloading
os.makedirs('/kaggle/working/vncorenlp', exist_ok=True)

# Download VnCoreNLP model
py_vncorenlp.download_model(save_dir='/kaggle/working/vncorenlp')

# Load the segmenter
rdrsegmenter = py_vncorenlp.VnCoreNLP(
    annotators=["wseg"], 
    save_dir='/kaggle/working/vncorenlp'
)

# Test word segmentation
text = "Ông Nguyễn Khắc Chúc đang làm việc tại Đại học Quốc gia Hà Nội. Bà Lan, vợ ông Chúc, cũng làm việc tại đây."
output = rdrsegmenter.word_segment(text)

print(output)

In [ ]:
# Map labels
label2id = {
    "gambling": 0, "movies": 1, "ecommerce": 2, "government": 3, "education": 4, "technology": 5,
    "tourism": 6, "health": 7, "finance": 8, "media": 9, "nonprofit": 10, "realestate": 11,
    "services": 12, "industries": 13, "agriculture": 14
}
id2label = {v: k for k, v in label2id.items()}

model_path = "insert-path"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

In [ ]:
def clean_text(text):
    # === Preserve domain dots, decimal dots, and URL hyphens ===
    text = re.sub(r'(\w)\.(?=\w)', r'\1<DOMAIN>', text)      # domain dots
    text = re.sub(r'(\d)\.(?=\d)', r'\1<DECIMAL>', text)     # decimal dots
    text = re.sub(r'(\w)-(?=\w)', r'\1<HYPHEN>', text)       # hyphen inside words/domains

    # === Remove remaining dots and hyphens ===
    text = text.replace('.', '')
    text = text.replace('-', '')

    # === Replace one or more underscores with a single space ===
    text = re.sub(r'_+', ' ', text)

    # === Restore preserved characters ===
    text = text.replace('<DOMAIN>', '.')
    text = text.replace('<DECIMAL>', '.')
    text = text.replace('<HYPHEN>', '-')

    # === Handle commas ===
    text = re.sub(r'(?<=[a-z0-9]),(?=[a-z])', ' ', text)  # digit/letter → letter
    text = re.sub(r'(?<=[a-z]),(?=[0-9])', ' ', text)     # letter → digit
    text = re.sub(r',(?=\D)|(?<=\D),', '', text)          # remove other commas

    # === Remove unwanted punctuation (keep quotes, %, /) ===
    text = re.sub(r'[^\w\s\.,/%"]', '', text)

    # === Normalize spaces ===
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [ ]:
def predict_batch(text_list):
    def preprocess(text):
        text = clean_text(text)
        segmented = rdrsegmenter.word_segment(text)
        return ' '.join(segmented)
    
    # Apply preprocessing
    processed_texts = [preprocess(text) for text in text_list]
    
    # Tokenize
    inputs = tokenizer(
        processed_texts,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=64
    )
    inputs = {key: value.to(device) for key, value in inputs.items()}
    # Inference
    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1)
    predicted_class_ids = torch.argmax(probabilities, dim=-1).cpu().numpy()
    predicted_labels = [id2label[idx] for idx in predicted_class_ids]

    return predicted_labels, probabilities.cpu().numpy()

In [ ]:
texts = [
    # "Onb🤑Huawei Store51.627.392",
    # "Onb🤑Huawei Store51.627.392 11 thg 7, 2025 — Onb   đặt chất lượng lên hàng đầu với tiêu chí: không gian chơi phải công bằng, thưởng phải đều, trò chơi phải đa dạng.",
    # "tặng code nhận kim cương❤️Đáng tin cậy v6.7.5 11 thg 7, 2025 — ️Trong phân khúc xe số thể thao phổ thông, Yamaha Exciter 155 VVA nổi bật với động cơ 155cc tích hợp công nghệ van biến thiên VVA, cho công suất ...",
    # "Công ty KO66💰Điện tử trực tuyến",
    # "chung cư dương nội webtretho 🧟 🇲🇽ba sòng bạc hàng ...",
    # "chung cư dương nội webtretho 🧟 🇲🇽ba sòng bạc hàng ... chung cư dương nội webtretho -Thông thường, các nhà thiết kế RFIC sử dụng các công cụ chuyên dụng để thực hiện xác minh tuần tự, đơn miền, ...",
    # "quán nhậu tá lả phạm văn đồng ua vé số trực tuyến",
    # "quán nhậu tá lả phạm văn đồng ua vé số trực tuyến quán nhậu tá lả phạm văn đồng -Thứ trưởng Nguyễn Bá Hoan cho biết, tháng 3 vừa qua, Bộ LĐ-TB-XH đã phối hợp cùng phía Nhật Bản tổ chức thành công kỳ thi ...",
    # "nạp lần đầu nhận gấp đôi-Điện tử trực tuyến",
    # "nạp lần đầu nhận gấp đôi-Điện tử trực tuyến 3 ngày trước — nạp lần đầu nhận gấp đôi   xây dựng uy tín từ chính sự công bằng, minh bạch và hệ thống phần thưởng tự động mỗi ngày.",
    # "tặng code nhận kim cương❤️Đáng tin cậy v6.7.5",
    # "tặng code nhận kim cương❤️Đáng tin cậy v6.7.5 11 thg 7, 2025 — ️Trong phân khúc xe số thể thao phổ thông, Yamaha Exciter 155 VVA nổi bật với động cơ 155cc tích hợp công nghệ van biến thiên VVA, cho công suất ...",
    # "KUWiN2 Com💷Huawei Store24.123.745",
    # "KUWiN2 Com💷Huawei Store24.123.745 4 thg 7, 2025 — Tại KUWiN2 Com  , mọi người đều có cơ hội như nhau. Hệ thống kiểm tra độc lập, thưởng ngày ổn định và kho game đa dạng khiến nơi đây trở ...",
    # "Nhà🤑Huawei Store51.627.392",
    # "Nhà🤑Huawei Store51.627.392 2 ngày trước — Nhà  Bạn đang tìm kiếm một nơi chơi game minh bạch, công bằng và thưởng thật mỗi ngày? Nhà   chính là sự lựa chọn được tin tưởng bởi cộng ...",
    # "sẽ",
    # "sẽ Cung cấp các dịch vụ và sản phẩm chất lượng của sẽ. Tận hưởng chất lượng và sự hài lòng từ sẽ.️",
    # "Khuyến mãi cực hot-Aptoide436.818.3",
    # "thôi rồi ta đã xa nhau",
    # "tặng code nhận thẻ nạp🤑App Store7.37.371",
    # "tặng code nhận kim cương❤️Đáng tin cậy v6.7.5",
    # "Xóc dĩa xanh chín-Điện tử trực tuyến",
    # "share bài viết nhận code-Huawei Store51.627.392",
    # "Tỉ lệ cược💷Huawei Store24.123.745",
    # "KUWiN2 Com💷Huawei Store24.123.745",
    "Sơ đồ website - UBND quận Cái Răng - Cần Thơ",
    "Sơ đồ website - UBND quận Cái Răng - Cần Thơ Thông tin tuyên truyền · Giới thiệu Lễ hội bánh dân gian nam bộ lần thứ XI năm 2024 · Thư kêu gọi hưởng ứng “Chiến dịch tuyên truyền, đấu tranh phòng, ...",
    "Đánh bạc qua mạng-Sòng bạc thông thường của Việt Nam",
    "Đánh bạc qua mạng-Sòng bạc thông thường của Việt Nam 7 ngày trước — Đánh bạc qua mạng  đem đến cho bạn trải nghiệm cá cược công bằng, được kiểm chứng minh bạch, cùng các trò chơi đa dạng từ kinh điển đến mới ...",
    "nhận hoàn trả mỗi ngày-Uy Tín và An Toàn",
    "nhận hoàn trả mỗi ngày-Uy Tín và An Toàn 10 thg 8, 2025 — ️Sáng 9/8, Thủ tướng Phạm Minh Chính đến kiểm tra, đôn đốc tiến độ thi công dự án Đường dây 500kV Lào Cai - Vĩnh Yên trên địa bàn tỉnh Phú Thọ.",
    "Thủ môn🔴Đảm bảo đánh giá v7.7.1",
    "3 cây tặng xu miễn phí",
    "tri ân định kỳ🔴Thương hiệu uy tín v7.3.9",
    "Cầu thủ bóng chuyền-Đăng ký sẽ tặng bạn 777K",
    "Sảnh người thật-Thương hiệu uy tín v7.3.9"
] 

batch_labels, batch_probs = predict_batch(texts)
for t, l, p in zip(texts, batch_labels, batch_probs):
    print(f"Text: {t}\nPredicted: {l}\n")